In [18]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

In [19]:
from config import g_key
gmaps.configure(api_key=g_key)

In [20]:
cities_csv = "../weatherpy/results/results.csv"

cities_df = pd.read_csv(cities_csv)

cities_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Hobart,75,AU,1651889741,91,-42.8794,147.3294,53.46,16.11
1,Qaanaaq,100,GL,1651889742,83,77.4840,-69.3632,13.08,3.60
2,Cherskiy,100,RU,1651889742,100,68.7500,161.3000,32.68,3.47
3,Praia da Vitória,31,PT,1651889549,85,38.7333,-27.0667,57.76,6.40
4,Cape Town,75,ZA,1651889703,88,-33.9258,18.4232,57.90,11.99


In [21]:
coords = cities_df[["Lat", "Lng"]]
humidity = cities_df['Humidity']
humidity_max = humidity.max()


# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(coords, 
                                 weights=humidity, 
                                 dissipating=False, 
                                 max_intensity=humidity_max,
                                 point_radius=3)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [22]:
temp_cities = cities_df.loc[(cities_df["Max Temp"] > 70) & (cities_df["Max Temp"] < 80) & (cities_df["Cloudiness"] == 0), :]
temp_cities = temp_cities.dropna()
temp_cities.reset_index(inplace=True)
del temp_cities['index']
temp_cities.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Cap Malheureux,0,MU,1651889748,68,-19.9842,57.6142,74.46,1.99
1,Geraldton,0,AU,1651889761,33,-28.7667,114.6000,73.81,19.57
2,Makkah al Mukarramah,0,SA,1651889540,38,21.4267,39.8261,73.63,7.31
3,Vila Velha,0,BR,1651889773,78,-20.3297,-40.2925,75.15,10.36
4,Ibrā’,0,OM,1651889792,62,22.6906,58.5334,74.77,0.45


In [23]:
hotels = []

for i in range(len(temp_cities)):
    lat = temp_cities.loc[i]['Lat']
    lng = temp_cities.loc[i]['Lng']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "lodging",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    jsn = requested.json()
    try:
        hotels.append(jsn['results'][0]['name'])
    except:
        hotels.append(np.nan)
temp_cities["Hotel Name"] = hotels
vaca_df = temp_cities.dropna()
vaca_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,Cap Malheureux,0,MU,1651889748,68,-19.9842,57.6142,74.46,1.99,Pereybere Hotel & Spa
1,Geraldton,0,AU,1651889761,33,-28.7667,114.6000,73.81,19.57,Broadwater Mariner Resort
2,Makkah al Mukarramah,0,SA,1651889540,38,21.4267,39.8261,73.63,7.31,"Makkah Clock Royal Tower, A Fairmont Hotel"
3,Vila Velha,0,BR,1651889773,78,-20.3297,-40.2925,75.15,10.36,Hotel Vitória Palace
4,Ibrā’,0,OM,1651889792,62,22.6906,58.5334,74.77,0.45,مزرعة راحة النفوس المنزفة


In [24]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in vaca_df.iterrows()]
locations = vaca_df[["Lat", "Lng"]]

In [25]:
fig = gmaps.figure()
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))